In [1]:
#encoding=utf-8
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
import torchvision
import torch.optim as optim
from torch.autograd import Variable

In [2]:
BATCH_SIZE=16
EPOCHES = 5
L_R = 0.01
HIDDEN_SIZE = 64

In [3]:
train_data = torchvision.datasets.MNIST(

    root='./mnist/',
    train=True,
    transform=torchvision.transforms.ToTensor(),
    download=False)

In [4]:
# print(train_data.size())
test_data = torchvision.datasets.MNIST(root='./mnist/',train=False)

train_loader = Data.DataLoader(dataset=train_data,batch_size=BATCH_SIZE,shuffle=True)
# test_loader = Data.DataLoader(dataset=test_data,batch_size=BATCH_SIZE,shuffle=True)#wrong
test_x = torch.unsqueeze(test_data.test_data,dim=1).type(torch.FloatTensor)[:3000]/255    #3000 samples
test_y = test_data.test_labels[:3000]

In [5]:
for i,(b_x,b_y) in enumerate(train_loader):
    print(b_x.squeeze(1).size())
    print(b_y)
    break

torch.Size([16, 28, 28])

 5
 1
 2
 0
 4
 0
 3
 4
 4
 7
 4
 8
 1
 1
 3
 0
[torch.LongTensor of size 16]



In [6]:
class net(nn.Module):
    def __init__(self,hidden_size,output_size):
        super(net,self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.rnn = nn.LSTM(28,hidden_size)
        self.out = nn.Linear(hidden_size,output_size)
    def forward(self,x_inputs):
        
        rnn_out,_ = self.rnn(x_inputs)
        rnn_out = rnn_out[-1].squeeze(0) #BATCH*HIDDEN_SIZE
        out = F.relu(self.out(rnn_out))
        return F.log_softmax(out)    
        

In [7]:
model = net(HIDDEN_SIZE,10)

In [8]:
optimizer = optim.Adam(model.parameters(),lr=L_R)
criterion = nn.NLLLoss()

In [9]:
for epoch in range(EPOCHES):
    model.train()
    total_loss = 0
    for i,(b_x,b_y) in enumerate(train_loader):
        loss = 0
        model.zero_grad()
        b_x = b_x.squeeze(1)
        b_x = Variable(b_x).transpose(0,1)
        b_y = Variable(b_y)
        output = model(b_x)
        loss = criterion(output,b_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.data[0]
        if i % 500 == 0:
            print('epoch:%d,step:%d,loss:%.4f'%(epoch,i,loss.data[-1]))
#         print('step:%d...loss:%.4f...'%(i,loss.data[0]))
    print('epoch:%d,total_loss:%.4f'%(epoch,total_loss))

epoch:0,step:0,loss:2.3030
epoch:0,step:500,loss:1.6864
epoch:0,step:1000,loss:0.4679
epoch:0,step:1500,loss:1.1451
epoch:0,step:2000,loss:0.4933
epoch:0,step:2500,loss:0.9470
epoch:0,step:3000,loss:1.0767
epoch:0,step:3500,loss:1.2987
epoch:0,total_loss:3938.0393
epoch:1,step:0,loss:1.0885
epoch:1,step:500,loss:0.7696
epoch:1,step:1000,loss:0.6904
epoch:1,step:1500,loss:0.7353
epoch:1,step:2000,loss:0.5828
epoch:1,step:2500,loss:0.4408
epoch:1,step:3000,loss:0.8845
epoch:1,step:3500,loss:1.0252
epoch:1,total_loss:3230.8967
epoch:2,step:0,loss:0.7710
epoch:2,step:500,loss:0.4623
epoch:2,step:1000,loss:0.3811
epoch:2,step:1500,loss:0.5873
epoch:2,step:2000,loss:0.4643
epoch:2,step:2500,loss:0.4542
epoch:2,step:3000,loss:0.3937
epoch:2,step:3500,loss:0.9160
epoch:2,total_loss:3151.6379
epoch:3,step:0,loss:0.8921
epoch:3,step:500,loss:0.5837
epoch:3,step:1000,loss:0.5658
epoch:3,step:1500,loss:0.4410
epoch:3,step:2000,loss:1.2964
epoch:3,step:2500,loss:0.7247
epoch:3,step:3000,loss:1.1018

In [10]:
model.eval()
total = 0
correct = 0
# output = model(Variable(test_x[0:10]))
# print(output)
import numpy as np
def random_batch(x,y,batch_size):
    len_data = len(x)
#     data ,y = np.array(x),np.array(y)
    batch_num = int(np.ceil(len_data/batch_size))
    shuffle_idx = np.random.permutation(np.arange(len_data))
    shuffle_data = x[shuffle_idx]
    shuffle_label = y[shuffle_idx]
    print(shuffle_data.size())
    for i in range(batch_num):
        start_idx = batch_size*i
        end_idx = min(batch_size*(i+1),len_data)
        yield shuffle_data[start_idx,end_idx],y


In [11]:
print(test_x.size())
print(test_y.size())

torch.Size([3000, 1, 28, 28])
torch.Size([3000])


In [12]:
total = 0
correct = 0
x_val = Variable(test_x.squeeze(1)[:16])
print(x_val.size())
ouput = model(x_val.transpose(0,1))
# print(output)
_,predict = torch.max(output.data,1)
print(predict.transpose(0,1))
print(test_y[:16].view(1,-1))
#     print(predict)
#     break
total += 16
print(total)
print(predict== test_y[:16]).contiguous()
print(torch.sum(predict== test_y[:16]))
correct += torch.sum(predict== test_y[:16])
print('test accury is:%.4f'%(correct*1.0/total))

torch.Size([16, 28, 28])


Columns 0 to 12 
    3     0     7     3     0     0     7     7     2     0     5     5     5

Columns 13 to 15 
    0     7     2
[torch.LongTensor of size 1x16]



Columns 0 to 12 
    7     2     1     0     4     1     4     9     5     9     0     6     9

Columns 13 to 15 
    0     1     5
[torch.LongTensor of size 1x16]

16

    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    1
    0
    0
[torch.ByteTensor of size 16x1]

1
test accury is:0.0625


In [13]:
for (b_x,b_y) in random_batch(test_x,test_y,BATCH_SIZE):
    b_x = Variable(b_x)
#     print(b_x.size())

    output = model(b_x)
    _,predict = torch.max(output,1)
#     print(predict)
#     break
    total += len(b_x)
    correct += (predict == b_y).float().sum()
print('test accury is:%.4f'%(correct/total))

TypeError: indexing a tensor with an object of type numpy.ndarray. The only supported types are integers, slices, numpy scalars and torch.LongTensor or torch.ByteTensor as the only argument.